<a href="https://colab.research.google.com/github/denisendirangu/CS_PROJECT1/blob/main/ICS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install earthengine-api

In [2]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-ndirangudenise61')

In [4]:
# Define the area of interest (AOI) and time range
aoi = ee.Geometry.Point([36.8219, -1.2921])  # Example point in Nairobi, Kenya
start_date = '2023-01-01'
end_date = '2023-01-31'

# Load Sentinel-1 SAR data
sentinel1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide (IW) mode is standard for Sentinel-1

# Retrieve the first image from the collection
sentinel1_image = sentinel1_collection.first()


In [6]:
# Load Sentinel-2 MSI data
sentinel2_collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # Limit cloud cover for clearer imagery

# Retrieve the first image from the collection
sentinel2_image = sentinel2_collection.first()


In [8]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    # Convert Earth Engine image to a tile layer for Folium
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom function to the folium Map class
folium.Map.add_ee_layer = add_ee_layer


In [9]:
# Define visualization parameters
sentinel1_vis_params = {
    'min': -25,
    'max': 0,
    'bands': ['VV']
}

# Display Sentinel-1 image
import folium
Map = folium.Map(location=[-1.2921, 36.8219], zoom_start=10)
Map.add_ee_layer(sentinel1_image, sentinel1_vis_params, 'Sentinel-1 VV')
Map


In [10]:
# Define visualization parameters
sentinel2_vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']  # True color (RGB)
}

# Display Sentinel-2 image
Map.add_ee_layer(sentinel2_image, sentinel2_vis_params, 'Sentinel-2 RGB')
Map


In [11]:
# Export Sentinel-1 data
export_sentinel1 = ee.batch.Export.image.toDrive(
    image=sentinel1_image.select('VV'),
    description='Sentinel1_VV',
    scale=10,
    region=aoi.buffer(5000).bounds(),  # Buffer the point for an area around it
    fileFormat='GeoTIFF'
)
export_sentinel1.start()


In [12]:
# Export Sentinel-2 data
export_sentinel2 = ee.batch.Export.image.toDrive(
    image=sentinel2_image.select(['B4', 'B3', 'B2']),  # RGB bands
    description='Sentinel2_RGB',
    scale=10,
    region=aoi.buffer(5000).bounds(),
    fileFormat='GeoTIFF'
)
export_sentinel2.start()


In [13]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


ModuleNotFoundError: No module named 'rasterio'